In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Reading and Preprocessing Dataset

In [4]:
# Read the dataset
dataPath = 'dataset/realAWSCloudwatch/realAWSCloudwatch/ec2_cpu_utilization_5f5533.csv'
data = pd.read_csv(dataPath)

In [8]:
# Main statistics
print(data.describe())

             value
count  4032.000000
mean     43.110372
std       4.303565
min      34.766000
25%      39.296000
50%      42.918000
75%      46.006000
max      68.092000


In [10]:
# Check for missing values
print(data.isnull().sum())

timestamp    0
value        0
dtype: int64


## Visualization